In [1]:
import pandas as pd

In [2]:
BTC_15 = pd.read_csv("data/data/ohlcv/BTCUSDT_15m_20170208_20250830.csv")
ETH_15 = pd.read_csv("data/data/ohlcv/ETHUSDT_15m_20170208_20250830.csv")

In [3]:
df = BTC_15.copy()
df = df[df["timestamp"] >= "2025-05-03"]

In [4]:
# Calculate the split index
split_idx = int(len(df) * 1)  # first 60% train, last 40% test

# Split into train/test
train = df.iloc[:split_idx]
test  = df.iloc[split_idx:]

print("Train size:", len(train))
print("Test size:", len(test))

Train size: 11473
Test size: 0


In [5]:
from Trading_algos.strategy_tester import StrategyTester

In [6]:
tester = StrategyTester(initial_capital=10000)

In [7]:
results = tester.run_backtest(
                            'FibonacciChannelStrategy',
                            train,
                            sensitivity=18,
                            leverage=1.0,
                            sl_percent=0,         # 1% adjustment to fib levels
                            fixed_stop=False,       # Use fib_786 (long) / fib_236 (short)
                            break_even_after_tp=1,  # Move to breakeven after TP2 hit
                            use_take_profits=True,
                            tp1_pct=1, tp1_close=40.0,
                            tp2_pct=2, tp2_close=30.0,
                            tp3_pct=3, tp3_close=20.0,
                            tp4_pct=4, tp4_close=10.0,
                            # Risk
                            risk_per_trade=0.015,   # 1.5% risk per trade
                            verbose=True
                        )

Starting backtest for FibonacciChannelStrategy
Data period: 2025-05-03 00:00:00+00:00 to 2025-08-30 12:00:00+00:00
Data points: 11473
Generating trading signals...
Signals — long: 29 short: 30
First long idx: [364, 1038, 1221] First short idx: [0, 921, 1164]
Running backtest simulation...


Backtesting: 100%|████████████████| 11473/11473 [00:08<00:00, 1409.27bars/s, Trades=47, Capital=$13,840, Position=FLAT]

Calculating performance metrics...

🎯 BACKTEST COMPLETED SUCCESSFULLY!
   Strategy: FibonacciChannelStrategy
   Total Trades: 47
   Final Return: 38.40%
   Final Capital: $13,839.65
   Win Rate: 72.34%
   Profit Factor: 3.76
   Max Drawdown: 3.87%
   Sharpe Ratio: 0.186
   Risk:Reward: 1.44
   Market Exposure: 727.74%
   Total Fees: $885.80

💡 Use display_comprehensive_metrics('FibonacciChannelStrategy') for detailed analysis


In [8]:
# Cell 4: View detailed metrics
tester.display_comprehensive_metrics('FibonacciChannelStrategy')

# Cell 5: Plot results
#tester.plot_results('SimpleMAStrategy', results, show_plot=True)


📊 COMPREHENSIVE METRICS TABLE FOR FIBONACCICHANNELSTRATEGY

PERFORMANCE:
--------------------------------------------------
  Total Return        : 38.40%
  Final Capital       : $13,839.65
  Total Trades        : 47
  Win Rate            : 25.53%
  Profit Factor       : 0.42
  Risk:Reward         : 0.45

RISK METRICS:
--------------------------------------------------
  Max Drawdown        : 3.87%
  Sharpe Ratio        : 0.186
  Sortino Ratio       : 0.483
  Calmar Ratio        : 4612.910
  Volatility          : 313.99%
  VaR (95%)           : -1.14%

TRADE ANALYSIS:
--------------------------------------------------
  Avg Win             : $63.36
  Avg Loss            : $139.91
  Largest Win         : $281.94
  Largest Loss        : $-201.69
  Max Consec Losses   : 3
  Max Consec Wins     : 7

STRATEGY BEHAVIOR:
--------------------------------------------------
  Trades/Day          : 0.40
  Avg Hold Time       : 31.06h
  Market Exposure     : 727.74%
  Break-even          : 6.57%
